## Simulator (fleet of uavs)

In [1]:
from simulator import Simulator
from simulator.config import DATA_PATH, Color
from simulator.helpers import clean
from simulator.helpers.coordinates import ENUPose, GRAPose
from simulator.planner import AutoPlan, Plan
from simulator.visualizer import (
              QGC,
              Gazebo,
              GazMarker,
              NoVisualizer,
              QGCMarker,
              SimVehicle,
)

clean()

## Simulation Positions

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5) for _ in base_homes]

## Create Vehicles

In [3]:
sysids = [1,2,3,4,5]
colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
vehs:list[SimVehicle] = []

for sysid, base_home, base_path, color in zip(sysids, base_homes, base_paths, colors):
    mission_path = DATA_PATH / f"mission_{sysid}.waypoints"
    plan = AutoPlan(
        name="simple_auto_plan",
        mission_path=str(mission_path),
    )
    plan.save_basic_mission_from_relative(
        sysid=sysid,
        gra_origin=gra_origin,
        relative_home=base_home,
        relative_path=base_path,
    )

    veh = SimVehicle.from_relative(
        sysid=sysid,
        gcs_name=f'Multicolor_{"".join([color.emoji for color in colors])}',
        plan=plan,
        color=color,
        enu_origin=enu_origin,
        relative_home=base_home,
        relative_path=base_path,
        model="iris",
    )
    
    vehs.append(veh)
    
    

## Visualizer

### Gazebo

In [4]:
gaz= Gazebo(gra_origin,world_path="simulator/gazebo/worlds/runway.world")
origin_gaz = GazMarker(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)
gaz.markers.append(origin_gaz)

### QGroundControl

In [5]:
qgc= QGC(gra_origin)
origin_qgc = QGCMarker(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)
qgc.markers.append(origin_qgc)

### No Visualizer

In [6]:
novis = NoVisualizer(gra_origin)

## Simulator

In [7]:
simulator = Simulator(
	visualizer=qgc,
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

simulator.show()

## Run

In [8]:
orac = simulator.launch()
orac.run()

00:39:46 - Oracle ⚪ - INFO - 🚀 GCS Multicolor_🟦🟩⬛🟧🟥 launched (PID 1440038)
00:39:46 - Oracle ⚪ - INFO - 🗺️  QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.
00:39:46 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 1 GCSs
00:41:34 - Oracle ⚪ - INFO - UAV 2 completed mission and exited
00:41:36 - Oracle ⚪ - INFO - UAV 1 completed mission and exited
00:41:36 - Oracle ⚪ - INFO - UAV 5 completed mission and exited
00:41:36 - Oracle ⚪ - INFO - UAV 4 completed mission and exited
00:41:36 - Oracle ⚪ - INFO - Received DONE from GCS Multicolor_🟦🟩⬛🟧🟥
00:41:36 - Oracle ⚪ - INFO - UAV 3 completed mission and exited
00:41:36 - Oracle ⚪ - INFO - ✅ All GCS threads completed
00:41:36 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
